In [ ]:
# Ingest EDGAR fossil-CO2-GHG data into Trino pipeline

In [ ]:
Run these in a notebook cell if you need to install onto your nb env

In [ ]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

In [ ]:
Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [21]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_iceberg_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'pcaf_sovereign_edgar'

In [23]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_iceberg_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [14]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
Open a Trino connection using JWT for authentication

In [6]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [8]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo',)
('company_data',)
('default',)
('defaultschema1',)
('demo',)
('eje_test_iceberg',)
('epa_frs',)
('epa_ghgrp',)
('epacems',)
('epacems_y95_al',)
('essd',)
('ghgrp_demo',)
('gleif',)
('gleif_mdt',)
('iceberg_demo',)
('information_schema',)
('ingest_schema',)
('iso3166',)
('itr_mdt',)
('metastore',)
('metastore_iceberg',)
('osc_corp_data',)
('pcaf_sovereign_footprint',)
('physical_risk_project',)
('pudl',)
('rmi_20210929',)
('rmi_20211120',)
('rmi_20220119',)
('rmi_utility_transition_hub',)
('sec_dera',)
('sfi_geoasset',)
('team1',)
('team2',)
('testaccessschema1',)
('testdb',)
('urgentem',)
('us_census',)
('wri',)
('wri_demo',)
('wri_dev',)
('wri_gppd',)
('wri_gppd_md',)
('wri_new',)
('wri_test',)


In [ ]:
Load GDP file (updated sporadically from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

In [7]:
import pandas as pd

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/EDGAR/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
ticker_file.download_file(f'/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
#ticker_df = pd.read_csv(f'/tmp/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2.csv',sep=",",skiprows=4)
#ticker_df
%run TransposeXLS.py --config Edgar.ini --output=EDGAR.csv 
df = pd.read_csv('EDGAR.csv')
#df = df[df['country_name'] == 'Germany']
#df_germany = df_germany[['data_provider','country_iso_code','validity_date','attribute','value']]
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value']].dropna(subset=['value'])
df["country_iso_code"]='###'
df.validity_date = df.validity_date.astype(str)
df = df.convert_dtypes()
print(df.info(verbose=True))
df
#df


Edgar.ini
file_list:
['/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls']
/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls
2
xls
/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021
['']
                    Country          1970          1971          1972  \
0               Afghanistan      1.718206      1.716487      1.712408   
1                   Albania      4.813550      4.813862      5.497650   
2                   Algeria     18.931573     22.315750     34.375052   
3                    Angola      8.952568      8.538305     10.388538   
4                  Anguilla      0.002168      0.002168      0.002264   
..                      ...           ...           ...           ...   
208  International Aviation    168.816953    168.816953    178.760913   
209  International Shipping    353.856240    353.856240    370.877710   
210                     NaN           NaN           NaN           NaN   
211                    EU27   3528.485156   3540.483237   3684.623393   
212   

,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value
0,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Afghanistan,1970,fossil_CO2_totals_by_country,1.718206
1,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Albania,1970,fossil_CO2_totals_by_country,4.81355
2,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Algeria,1970,fossil_CO2_totals_by_country,18.931573
3,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Angola,1970,fossil_CO2_totals_by_country,8.952568
4,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Anguilla,1970,fossil_CO2_totals_by_country,0.002168
...,...,...,...,...,...,...,...
95965,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Yemen,2020,fossil_CO2_per_GDP_by_country,0.206664
95966,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Zambia,2020,fossil_CO2_per_GDP_by_country,0.124774
95967,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Zimbabwe,2020,fossil_CO2_per_GDP_by_country,0.283339
95969,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,EU27,2020,fossil_CO2_per_GDP_by_country,0.141057


In [15]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())



drop table if exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_values

[(True,)]


In [16]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())


create table if not exists osc_datacommons_iceberg_dev.pcaf_sovereign_footprint.pcaf_sovereign_values(
    rec_source varchar,
    data_provider varchar,
    country_iso_code varchar,
    country_name varchar,
    validity_date varchar,
    attribute varchar,
    value double
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)

[(True,)]


In [ ]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

In [17]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value


In [24]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
#df["country_iso_code"]='###'
#df.validity_date = df.validity_date.astype(str)
df.to_sql(ingest_table,
           con=engine, 
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 1000, verbose = True))

osc_datacommons_iceberg_dev
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sovereign_edgar"
batch insert result: [(1000,)]
inserting 1000 records
constructed fully qualified table name as: "pcaf_sovereign_footprint.pcaf_sover

In [34]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}" + "where country_name='Germany' and validity_date='2020'"""
pd.read_sql(sql, engine)


,rec_source,data_provider,country_iso_code,country_name,validity_date,attribute,value
0,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_by_sector_and_countr(Power Industry),210.139514
1,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_by_sector_and_countr(Buildings),126.425378
2,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_by_sector_and_countr(Transport),142.123944
3,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_by_sector_and_countr(Other industri...,101.627986
4,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_by_sector_and_countr(Other sectors),56.559642
5,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_per_capita_by_countr,7.715932
6,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_per_GDP_by_country,0.150249
7,EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls,EDGAR,###,Germany,2020,fossil_CO2_totals_by_country,636.876464
